In [1]:
from math import sqrt
import fiona  # pour lire les shapefile
import pyproj
import numpy as np
import pandas as pd

In [2]:
lambert=pyproj.Proj("+init=EPSG:3035")
geo_grid = pd.read_csv('data/geostat/GEOSTAT_grid_POP_1K_2011_V2_0_1.csv')

/home/savinien/anaconda3/envs/py35_1/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
geo_grid2 = pd.read_csv('data/geostat/JRC-GHSL_AIT-grid-POP_1K_2011.csv')
subframeLUX = geo_grid2[geo_grid2.CNTR_CODE == 'LU']
geo_grid_merged = pd.concat([geo_grid, subframeLUX])

In [4]:
import tarfile

In [5]:
with tarfile.open("data/geostat/GEOSTATReferenceGrid/Grid_ETRS89_LAEA_1K-ref_GEOSTAT_POP_2011_V2_0_1.shp.tar.gz") as tar:
    tar.extractall(path="data/geostat/GEOSTATReferenceGrid/")

In [6]:
with tarfile.open("data/geostat/GEOSTATReferenceGrid/Grid_ETRS89_LAEA_1K-ref_GEOSTAT_POP_2011_V2_0_1.dbf.tar.gz") as tar:
    tar.extractall(path="data/geostat/GEOSTATReferenceGrid/")

In [7]:
x_laea = []
y_laea = []
grid_shape = []

with fiona.open('data/geostat/GEOSTATReferenceGrid/Grid_ETRS89_LAEA_1K-ref_GEOSTAT_POP_2011_V2_0_1.shp') as shapeFile:
    for i, point in enumerate(shapeFile):
        x_laea.append(point['geometry']['coordinates'][0][0][0])
        y_laea.append(point['geometry']['coordinates'][0][0][1])
        grid_shape.append(str(point['properties']['GRD_ID']))

In [8]:
df_shapefiles = pd.DataFrame({'x':x_laea, 'y':y_laea, 'grid':grid_shape})

In [9]:
df_merged = (pd.merge(df_shapefiles, geo_grid_merged[['TOT_P','GRD_ID','CNTR_CODE']], how='left', left_on='grid', right_on='GRD_ID')
 .drop('GRD_ID', axis=1))

In [10]:
df_merged.to_csv('data/grids1km_pop_lambert.csv')

In [11]:
df_merged.to_pickle('data/grids1km_pop_lambert.pkl')

In [12]:
longitudes=[2.875,-0.69083,5.27083,6.21806,0.17028,4.79056,0.6528,4.75667,2.51667,7.563036,-1.88167,0.84528,2.135,3.51778,0.63528,1.21194,4.75528,1.58349,4.72249]
latitudes=[47.50972,45.25611,45.79833,49.41583,47.2306,50.09,46.45667,44.63306,47.73306,47.903108,49.53639,44.10667,51.01444,48.51528,49.85778,49.97611,45.40444,47.723982,44.335698]
nomsCentrales=['Belleville','Blayais','Bugey','Cattenom','Chinon-B','Chooz-B','Civaux','Cruas','Dampierre','Fessenheim','Flamanville','Golfech','Gravelines','Nogent','Paluel','Penly','Saint-Alban','St. Laurent','Tricastin']

In [13]:
df_centrales = pd.DataFrame({'longitudes':longitudes, 'latitudes':latitudes, 'nomsCentrales':nomsCentrales})

In [14]:
df_centrales[['x', 'y']] = df_centrales[['longitudes', 'latitudes']].apply(lambda row: list(lambert(row[0],row[1])), axis=1)

In [15]:
def return_pop(row, km):
    mask = ((df_merged.x+500-row.x)**2
            +(df_merged.y+500-row.y)**2).apply(sqrt)< (1000*km)
    return df_merged[mask].TOT_P.sum()

In [16]:
def search_pop(df, distance):
    df['pop_' + str(distance) + '_km'] = df_centrales.apply(return_pop, args=(distance,), axis=1)
    return df

In [17]:
search_pop(df_centrales, 10)
search_pop(df_centrales, 20)
search_pop(df_centrales, 30)
search_pop(df_centrales, 100)

,latitudes,longitudes,nomsCentrales,x,y,pop_10_km,pop_20_km,pop_30_km,pop_100_km
0,47.509720,2.875000,Belleville,3.784859e+06,2.736510e+06,12736.0,50815.0,97251.0,1738957.0
1,45.256110,-0.690830,Blayais,3.483758e+06,2.520596e+06,15581.0,67348.0,148757.0,2276468.0
2,45.798330,5.270830,Bugey,3.953121e+06,2.532309e+06,42646.0,189795.0,705931.0,6137194.0
3,49.415830,6.218060,Cattenom,4.046662e+06,2.929604e+06,92989.0,388704.0,813932.0,4426623.0
4,47.230600,0.170280,Chinon-B,3.578537e+06,2.729031e+06,29505.0,86334.0,171913.0,2719971.0
5,50.090000,4.790560,Chooz-B,3.948525e+06,3.010752e+06,22972.0,70071.0,182047.0,6590937.0
6,46.456670,0.652800,Civaux,3.604317e+06,2.638887e+06,11950.0,50061.0,206764.0,1798093.0
7,44.633060,4.756670,Cruas,3.904536e+06,2.405940e+06,52694.0,139678.0,261372.0,3195321.0
8,47.733060,2.516670,Dampierre,3.760433e+06,2.763887e+06,17727.0,61923.0,125007.0,2538767.0
9,47.903108,7.563036,Fessenheim,4.138736e+06,2.757394e+06,50346.0,241690.0,962487.0,7442318.0
